In [124]:
import urllib.request
from lxml import html
import os
import pandas as pd
url_rawdata = 'ftp://ftp.ebi.ac.uk/pub/databases/intact/current/psimitab/intact.txt'

histone_genes_list_old = 'HIST1H2BB HIST1H2BC HIST1H2BE HIST1H2BF HIST1H2BG HIST1H2BI HIST1H2BD HIST1H2BH HIST1H2BJ HIST1H2BK HIST1H2BL \
           HIST1H2BM HIST1H2BN HIST1H2BO HIST2H2BE HIST2H2BF HIST3H2BB H2BFS H2BFM H2BFWT HIST1H2BA HIST1H2AG HIST1H2AI \
           HIST1H2AK HIST1H2AL HIST1H2AM HIST1H2AC HIST1H2AD HIST1H2AE HIST1H2AH HIST1H2AJ HIST2H2AB HIST2H2AC HIST3H2A \
           HIST1H2APS4 H2AFJ H2AFX H2AFZ H2AFV H2AFY H2AFY2 HIST1H2AA H2AFB1 H2AFB2 H2AFB3 HYPM HIST1H3A HIST1H3B HIST1H3C \
           HIST1H3D HIST1H3E HIST1H3F HIST1H3G HIST1H3H HIST1H3I HIST1H3J HIST2H3A HIST2H3C HIST2H3D HIST2H3PS2 H3F3A H3F3B \
           H3F3C CENPA HIST3H3 HIST1H4A HIST1H4B HIST1H4C HIST1H4D HIST1H4E HIST1H4F HIST1H4H HIST1H4I HIST1H4J HIST1H4K HIST1H4L HIST2H4A HIST2H4B HIST4H4'
histone_genes_list_old = histone_genes_list_old.split()
histone_genes_list_old = [x.lower() for x in histone_genes_list_old]

In [221]:
from ftplib import FTP
from dateutil import parser

previous_release_date = parser.parse("12/25/2030")
ftp = FTP()
ftp.connect('ftp.ebi.ac.uk')
ftp.login()
ftp.cwd('/pub/databases/intact/current/psimitab/')
lines = []
ftp.dir(".", lines.append)

for line in lines:
    tokens = line.split(maxsplit = 9)
    name = tokens[8]
    time_str = tokens[5] + " " + tokens[6] + " " + tokens[7]
    time = parser.parse(time_str)
    if name == 'intact.txt':
        latest_release_date = time
        
if latest_release_date > previous_release_date:
    previous_release_date = latest_release_date
    url_list = 'ftp://ftp.ebi.ac.uk/pub/databases/intact/current/psimitab/intact.txt'
    urllib.request.urlretrieve(url_rawdata, 'rawdata.txt')

print(previous_release_date,latest_release_date)

2030-12-25 00:00:00 2019-08-29 09:35:00


In [116]:
histone_genes_table = pd.read_csv('histone_genes.csv', sep=',', low_memory=False)
histone_genes_list = histone_genes_table.loc[:,'HGNC Symbol'].tolist()
histone_genes_list = [x.lower() for x in histone_genes_list if str(x) != 'nan']
rawdata = pd.read_csv('rawdata.txt', sep='\t', low_memory=False)

In [117]:
rawdata['Alias(es) interactor B'] = rawdata['Alias(es) interactor B'].str.lower()
rawdata['Alias(es) interactor A'] = rawdata['Alias(es) interactor A'].str.lower()


In [125]:
boolean=rawdata['Alias(es) interactor A'].str.contains('|'.join(histone_genes_list_old))|rawdata['Alias(es) interactor B'].str.contains('|'.join(histone_genes_list_old))
rawdata1 = rawdata[boolean]

In [119]:
rawdata1 = rawdata1[rawdata1['Interaction type(s)'].str.contains('direct interaction|physical association')]